In [2]:
%matplotlib inline

Define path to data: (It's a good idea to put it in a subdirectory of your notebooks folder, and then exclude that directory from git control by adding it to .gitignore.)

In [11]:
path = "data/dogscats-redux/"
#path = "data/dogscats-redux/sample/"

A few basic libraries that we'll need for the initial exercises:

In [2]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

We have created a file most imaginatively called 'utils.py' to store any little convenience functions we'll want to use. We will discuss these as we use them.

In [3]:
import utils; reload(utils)
from utils import plots

Using cuDNN version 5110 on context None
Preallocating 6553/8192 Mb (0.800000) on cuda0
Mapped name None to device cuda0: GeForce GTX 1080 (0000:01:00.0)
Using Theano backend.


In [4]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64

In [5]:
# Import our class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [12]:
vgg = Vgg16()
# Grab a few images at a time for training and validation.
# NB: They must be in subdirectories named based on their category
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/1
23000/23000 [==============================] - 189s - loss: 0.1213 - acc: 0.9696 - val_loss: 0.0766 - val_acc: 0.9800


In [87]:
vgg.fit(batches, val_batches, nb_epoch=5)

Epoch 1/5
23000/23000 [==============================] - 190s - loss: 0.1027 - acc: 0.9787 - val_loss: 0.1071 - val_acc: 0.9780
Epoch 2/5
23000/23000 [==============================] - 183s - loss: 0.0959 - acc: 0.9791 - val_loss: 0.0702 - val_acc: 0.9860
Epoch 3/5
23000/23000 [==============================] - 183s - loss: 0.0953 - acc: 0.9796 - val_loss: 0.1035 - val_acc: 0.9800
Epoch 4/5
23000/23000 [==============================] - 185s - loss: 0.1004 - acc: 0.9803 - val_loss: 0.0576 - val_acc: 0.9875
Epoch 5/5
23000/23000 [==============================] - 186s - loss: 0.1042 - acc: 0.9810 - val_loss: 0.1073 - val_acc: 0.9830


In [88]:
# We can train on the old data, too!
old_path = 'data/dogscats/'
batches = vgg.get_batches(old_path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(old_path+'valid', batch_size=batch_size*2)
vgg.fit(batches, val_batches, nb_epoch=1)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/1
23000/23000 [==============================] - 249s - loss: 0.1091 - acc: 0.9809 - val_loss: 0.0603 - val_acc: 0.9875


In [27]:
def format_answer(prediction):
    cat_chance, dog_chance = prediction
    if dog_chance > cat_chance:
        return dog_chance
    elif cat_chance > dog_chance:
        return 1 - cat_chance        
    else:
        return 0.5

In [89]:
from keras.preprocessing.image import load_img, img_to_array

with open(path+'submission2.csv', 'w') as csv:
    csv.write("id,label\n")
    
    for i in xrange(1, 12501):
        img = load_img(path+'test/unknown/'+str(i)+'.jpg', target_size=(224, 224))
        img_arr = img_to_array(img)
        predictions = vgg.model.predict(np.array([img_arr]))
        csv.write("%d,%f\n" % (i, format_answer(predictions[0])))